In [1]:
# imports
import pandas as pd

In [2]:
# load in data
df = pd.read_csv('/Users/josephbell/Documents/COVID-19_Case_Surveillance_Public_Use_Data.csv')
print(df.shape)
df.head()

/Users/josephbell/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(5760066, 11)


,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn
0,2020/06/30,NaN,NaN,Probable Case,Other,0 - 9 Years,Unknown,Unknown,Unknown,No,Unknown
1,2020/06/30,NaN,NaN,Probable Case,Other,0 - 9 Years,Unknown,Unknown,Unknown,No,Unknown
2,2020/07/05,NaN,NaN,Laboratory-confirmed case,Other,0 - 9 Years,Unknown,Missing,Missing,Missing,Missing
3,2020/06/10,NaN,2020/03/27,Probable Case,Other,0 - 9 Years,Unknown,No,Unknown,No,Unknown
4,2020/07/13,NaN,2020/04/20,Probable Case,Other,0 - 9 Years,Unknown,No,Unknown,No,Unknown


In [3]:
df['death_yn'].value_counts()

No         2662955
Missing    2321809
Unknown     617047
Yes         158255
Name: death_yn, dtype: int64

In [4]:
df['medcond_yn'].value_counts()

Missing    3976510
Yes         647375
Unknown     634512
No          501669
Name: medcond_yn, dtype: int64

In [5]:
count = df.groupby(['age_group', 'death_yn']).count()
count['count'] = count['cdc_report_dt']
count = count.drop(columns=['cdc_report_dt', 'pos_spec_dt', 'onset_dt', 'current_status', 'sex',
       'Race and ethnicity (combined)', 'hosp_yn', 'icu_yn', 'medcond_yn'])
count

count
age_group     death_yn        
0 - 9 Years   Missing    71676
              No        100117
              Unknown    17582
              Yes           55
10 - 19 Years Missing   214867
              No        270695
              Unknown    59379
              Yes           95
20 - 29 Years Missing   463612
              No        533316
              Unknown   130201
              Yes          720
30 - 39 Years Missing   395426
              No        447131
              Unknown    99832
              Yes         1976
40 - 49 Years Missing   365776
              No        413671
              Unknown    93516
              Yes         4798
50 - 59 Years Missing   353996
              No        392682
              Unknown    90938
              Yes        12409
60 - 69 Years Missing   239445
              No        267322
              Unknown    62428
              Yes        26292
70 - 79 Years Missing   123586
              No        140717
              Unknown    33836
              Yes        38943
80+ Years     Missing    88020
              No         95357
              Unknown    28650
              Yes        72946
Unknown       Missing     5368
              No          1903
              Unknown      683
              Yes           12

In [6]:
count = count.reset_index()

In [7]:
def label_total_deaths(row):
    if row['death_yn'] == 'Yes':
        return '158222'
    else:
        return '0'

count['total_deaths'] = count.apply(lambda row: label_total_deaths(row), axis=1)
count['total_deaths'] = count['total_deaths'].astype(int)
count['age_group_percent_deaths'] = count['count'] / count['total_deaths']
count.head()

,age_group,death_yn,count,total_deaths,age_group_percent_deaths
0,0 - 9 Years,Missing,71676,0,inf
1,0 - 9 Years,No,100117,0,inf
2,0 - 9 Years,Unknown,17582,0,inf
3,0 - 9 Years,Yes,55,158222,0.000348
4,10 - 19 Years,Missing,214867,0,inf


In [8]:
count[count['death_yn'] == 'Yes']

,age_group,death_yn,count,total_deaths,age_group_percent_deaths
3,0 - 9 Years,Yes,55,158222,0.000348
7,10 - 19 Years,Yes,95,158222,0.000600
11,20 - 29 Years,Yes,720,158222,0.004551
15,30 - 39 Years,Yes,1976,158222,0.012489
19,40 - 49 Years,Yes,4798,158222,0.030324
23,50 - 59 Years,Yes,12409,158222,0.078428
27,60 - 69 Years,Yes,26292,158222,0.166172
31,70 - 79 Years,Yes,38943,158222,0.246129
35,80+ Years,Yes,72946,158222,0.461036
39,Unknown,Yes,12,158222,0.000076


In [9]:
# TODO - make a new column that counts medcond_yn and death_yn
def label_row(row):
    if row['death_yn'] == 'Yes' and row['medcond_yn'] == 'Yes':
        return 'Yes'
    elif row['death_yn'] == 'Yes' and row['medcond_yn'] == 'No':
        return 'No'
    else:
        return 'Unknown or Missing'

df['death_medcond_yn'] = df.apply(lambda row: label_row(row), axis=1)
df.head()

,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn,death_medcond_yn
0,2020/06/30,NaN,NaN,Probable Case,Other,0 - 9 Years,Unknown,Unknown,Unknown,No,Unknown,Unknown or Missing
1,2020/06/30,NaN,NaN,Probable Case,Other,0 - 9 Years,Unknown,Unknown,Unknown,No,Unknown,Unknown or Missing
2,2020/07/05,NaN,NaN,Laboratory-confirmed case,Other,0 - 9 Years,Unknown,Missing,Missing,Missing,Missing,Unknown or Missing
3,2020/06/10,NaN,2020/03/27,Probable Case,Other,0 - 9 Years,Unknown,No,Unknown,No,Unknown,Unknown or Missing
4,2020/07/13,NaN,2020/04/20,Probable Case,Other,0 - 9 Years,Unknown,No,Unknown,No,Unknown,Unknown or Missing


In [10]:
df['death_medcond_yn'].value_counts()[1] / df['death_yn'].value_counts()[3]

0.4296609901740861

In [11]:
df['death_medcond_yn'].value_counts()[2] / df['death_yn'].value_counts()[3]

0.018166882562952198